## About Dataset

Foursquare is a social media platform where users can share their locations and rate venues. This dataset contains 09/2013's values.

You need to download the dataset with torrent to access it.

> https://archive.org/details/201309_foursquare_dataset_umn

<img src="images/raw_dataset.png">

## Accessing data from file system and cleaning it

```PostgreSQL
\COPY "ODS"."EXT_FS_VENUES" FROM 'C:/Users/uguraskar/Downloads/201309_foursquare_dataset_umn/fsq/umn_foursquare_datasets/venues.dat' WITH (FORMAT CSV, DELIMITER ',', FORCE_NULL(latitude,longitude));
```

In [45]:
import time
import math
import os
import re

source_dir = os.getcwd()
raw_data_folder = source_dir+"\\data\\raw"
clean_data_folder = source_dir+"\\data\\clean"

for filename in os.listdir(raw_data_folder):
    with open(os.path.join(raw_data_folder, filename), 'r') as f: # open in readonly mode
        start_time = math.trunc(time.time())
        clean_file = open(os.path.join(clean_data_folder,filename),"w")
        line_number = 0
        for line in f:
            line_number += 1 #Don't carry this to end of the elif statements otherwise line number will be zero indefinitely
            if(line_number in range(1,3)):
                #print("Deleted line: {line_number}".format(line_number=line_number))
                continue
            elif(line.endswith("rows)\n")):
                break
            else:
                clean_format = line.replace(" ","").replace("|",";")
                clean_file.write(clean_format)
        clean_file.close()
        end_time = math.trunc(time.time())
        print("Cleaned and created {filename} in {execute_time} seconds".format(execute_time=end_time-start_time, filename=filename))

Cleaned and created checkins.dat in 3 seconds
Cleaned and created ratings.dat in 6 seconds
Cleaned and created socialgraph.dat in 64 seconds
Cleaned and created users.dat in 6 seconds
Cleaned and created venues.dat in 3 seconds


## Sources:

 1. [Dataset](https://archive.org/details/201309_foursquare_dataset_umn)

 2. [How to open every file in a folder?](https://stackoverflow.com/questions/18262293/how-to-open-every-file-in-a-folder)

 3. [Python- reading a file line by line and processing](https://stackoverflow.com/questions/53749062/python-reading-a-file-line-by-line-and-processing)

